## French storm tracks derived from ERA5 wind gust data for the most impactful events from the late 20th to early 21st century

In [1]:
import os
import geopandas as gpd
from ipyleaflet import (
    Map, Polyline, Polygon, CircleMarker,
    LayerGroup, LayersControl, basemaps, basemap_to_tiles, Popup
)
from ipywidgets import HTML, Layout

# Set folder path to your shapefiles
folder_path = "shapefiles"


# Create the map 
m = Map(
    center=[46.5, -1.5],
    zoom=6,
    basemap=basemap_to_tiles(basemaps.Esri.WorldImagery),
    scroll_wheel_zoom=True,
    layout=Layout(width="100%", height="95vh")
)

# Create layer groups
trajectory_layer = LayerGroup(name="Trajectories")
surface_layer = LayerGroup(name="Surfaces")
centroid_layer = LayerGroup(name="Centroids")

# Storm name mapping
storm_name_dict = {
    "03_to_03_021990_10m_wind_gust_V24.7_S50_MODIF": "Herta 1990",
    "11_to_11_021974_10m_wind_gust_V24.7_S50_MODIF": "February storm 1974",
    "15_to_16_101987_10m_wind_gust_V24.7_S50_MODIF": "Great storm of 1987",
    "25_to_25_011990_10m_wind_gust_V24.7_S50_MODIF": "Daria 1990",
    "26_to_26_121999_10m_wind_gust_V24.7_S50_MODIF": "Lothar 1999",
    "27_to_28_121999_10m_wind_gust_V24.7_S50_MODIF": "Martin 1999",
    "28_to_28_022010_10m_wind_gust_V24.7_S50": "Xynthia 2010"
}

# High-contrast color palette
storm_colors = [
    "#FF0000", "#00FFFF", "#FFFF00", "#00FF00", "#FFA500",
    "#FF00FF", "#00BFFF", "#ADFF2F", "#FF1493", "#7CFC00"
]

# Group shapefiles by storm ID
def get_shapefile_groups():
    grouped = {}
    for f in os.listdir(folder_path):
        if f.endswith(".shp"):
            prefix = f[:3]
            key = f[3:].split(".shp")[0]  # Unique part after prefix
            grouped.setdefault(key, {})[prefix] = f
    return grouped

# Assign storm colors based on order
storm_ids_sorted = sorted(get_shapefile_groups().keys())
color_dict = {storm_id: storm_colors[i % len(storm_colors)] for i, storm_id in enumerate(storm_ids_sorted)}

# Load and display all storms
for storm_id, files in get_shapefile_groups().items():
    storm_display_name = storm_name_dict.get(storm_id, storm_id)
    color = color_dict.get(storm_id, "#FF0000") 

    # Add trajectory
    if "WT_" in files:
        gdf = gpd.read_file(os.path.join(folder_path, files["WT_"]))
        for _, row in gdf.iterrows():
            coords = list(row.geometry.coords)
            polyline = Polyline(
                locations=[(lat, lon) for lon, lat in coords],
                color=color,
                weight=2,
                opacity=1.0,
                fill=False
            )
            trajectory_layer.add_layer(polyline)

    # Add surface
    if "WS_" in files:
        gdf = gpd.read_file(os.path.join(folder_path, files["WS_"]))
        for _, row in gdf.iterrows():
            coords = list(row.geometry.exterior.coords)
            polygon = Polygon(
                locations=[(lat, lon) for lon, lat in coords],
                color=color,
                fill_color=color,
                weight=1,
                opacity=0.5
            )
            surface_layer.add_layer(polygon)

    # Add centroids
    if "WC_" in files:
        gdf = gpd.read_file(os.path.join(folder_path, files["WC_"]))
        for _, row in gdf.iterrows():
            location = (row.geometry.y, row.geometry.x)
            marker = CircleMarker(
                location=location,
                radius=1,
                color=color,
                fill_color=color,
                fill_opacity=1,
                draggable=False
            )
            popup_content = HTML(
                f"<b>Storm:</b> {storm_display_name}<br><b>Datetime:</b> {row.get('datetime', 'N/A')}<br><b>Location:</b> {location}"
            )
            marker.popup = popup_content
            centroid_layer.add_layer(marker)

# Add layers to map
m.add_layer(surface_layer)
m.add_layer(trajectory_layer)
m.add_layer(centroid_layer)
m.add_control(LayersControl(position="topright"))

# Display the map
m


Map(center=[46.5, -1.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…